In [150]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import time
import pydde as d

In [151]:
#Parameters
samplenum = 3
batchsize = samplenum
epochs = 10
hiddenlayers = [200,100]
input_size = 3
output_size = 180
learning_rate = 0.01
time_length = 60; #seconds

In [152]:
# Generate simulation
sim = d.PySimSeq('test2.sim', 60)
yseq = sim.compute(sim.p)
f = sim.f(sim.y, sim.ydot, sim.yddot, sim.p)
df = sim.df_dp(sim.y, sim.ydot, sim.yddot, sim.p)
dy = sim.dy_dp(sim.y, sim.ydot, sim.yddot, sim.p)


In [153]:
#Sample targets only variables in z direction
x = np.zeros((samplenum,3))
x[:,2] = np.random.rand(samplenum)
#x[:,0] = np.random.rand(samplenum)
x[:,1] = 2

In [154]:
#Sample ytraj
start_time1 = time.time()
p = np.zeros((samplenum, 3*time_length))
y = np.zeros((samplenum, 3*time_length))

for i in range(samplenum):
    p[i, :] = sim.sample_ptraj(x[i, :], sim.p)
    y[i, :] = sim.compute(p[i, :].transpose())

print(f'\nDuration: {(time.time() - start_time1)/60:.3f} min') # print the time elapsed


Duration: 0.090 min


In [155]:
#Preprocesing
x= torch.tensor(x)
y= torch.tensor(y)

## Building the custon Simulation Activation Function

In [161]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input, y_target):
        #print(input.shape)
        #print(y_target.shape)
        #ctx.mark_dirty( y_target)
        p = input.clone().numpy().transpose()
        y_target = y_target.clone().numpy()
        #print(p.shape)
        #print(y_target.shape)
        yTraj_pred = np.zeros((len(y_target[:, 1]),180))
        
        for i in range(len(y_target[:,1])):
            yTraj_pred[i, :] = sim.compute(p[:, i])

        yTraj_pred = torch.tensor(yTraj_pred)
        #print(f'yTraj_pred {yTraj_pred.shape}')
        
        ctx.save_for_backward(input, yTraj_pred)
        
        return yTraj_pred, input
    
    @staticmethod
    def backward(ctx, grad_output, input):
        input, yTraj_pred = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        yTraj_pred = yTraj_pred.clone().numpy()
        dy_dp = np.zeros((samplenum, 3*time_length))
        for i in range(len(yTraj_pred[:,1])):
            yTraj_pred[i, :] = sim.compute(p[:, i])
            dy_dp = sim.dy_dp(sim.y, sim.ydot, sim.yddot, p[:, i])
        #print(dy_dp)
        
        torch.tensor(dy_dp).mm(grad_output.t()).t()
        return grad_input, None

Simulate = Simulate.apply

## Building the Model

In [162]:
class ActiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(ActiveLearn, self).__init__()

        self.L_in = nn.Linear(input_size, output_size)
        self.P = nn.Linear(output_size, output_size)
        self.Relu = nn.ReLU(inplace=True)
    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.P(x)
        x = self.Relu(x)
        x, p = Simulate(x, input)
        return x, p


In [163]:
model = ActiveLearn(input_size, output_size)

criterion = nn.MSELoss()  # RMSE = np.sqrt(MSE)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

x_train = x.float()
y_train = y.float()

## Train the model

In [164]:
torch.autograd.set_detect_anomaly(True)
start_time = time.time()

losses = []

for i in range(epochs):
    y_pred, p_pred = model(x_train)

    #loss = torch.sqrt(criterion(y_pred, y)) # RMSE
    loss = criterion(y_pred, y) # RMSE
    losses.append(loss)

    if i%10 == 1:
        print(f'epoch: {i:3}/{epochs}  loss: {loss.item():10.8f}')

    #Clear the gradient buffer (w <-- w - lr*gradient)
    optimizer.zero_grad()
    #Back Prop
    loss.backward()
    optimizer.step()
    i+=1

print(f'epoch: {i:3} loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {(time.time() - start_time1)/60:.3f} min') # print the time elapsed

RuntimeError: Function SimulateBackward returned an invalid gradient at index 0 - got [180, 180] but expected shape compatible with [3, 180]

## Grad Check

In [146]:
from torch.autograd import gradcheck

# gradcheck takes a tuple of tensors as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
input = (x_train, y_train)
test = gradcheck(Simulate, input, eps=1e-6, atol=1e-4)
print(test)

ValueError: gradcheck expects at least one input tensor to require gradient, but none of the them have requires_grad=True.

In [160]:
model.layer[0].weight

AttributeError: 'ActiveLearn' object has no attribute 'layer'